In [1]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!python -m spacy download de_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 5.4 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [3]:
import os
import glob
import nemo.collections.asr as nemo_asr
import torch
import numpy as np
import ctc_decoders
import torchaudio
import soundfile as sf
import librosa
import pandas as pd

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [4]:
from processing.text_processor import preprocess_transcript_for_alignment, preprocess_transcript_for_sentence_split, split_to_sentences
from transcribing.stt_transcribe import get_stt_transcription, get_stt_probs

In [5]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(device)

1.11.0
cuda


## Read true transcript
Read the text from the complete protocol provided by the parliament of canton Schwyz. After reading the text process everything and split the whole text into its sentences. 

In [6]:
PATH_TO_TRANSCRIPT = 'data/text_processed_2021-04-28.txt'

In [7]:
with open(PATH_TO_TRANSCRIPT, encoding='utf-8') as f:
    transcript = f.read()
    transcript = preprocess_transcript_for_sentence_split(transcript)
    truth_sentences = split_to_sentences(transcript)

In [8]:
#print(truth_sentences)

## Retrieve all audio files to later match with the corresponding text

In [9]:
PATH_TO_AUDIO = '/data/voice/SZ_parliament/raw/parts'

In [10]:
audio_files = glob.glob(os.path.join(PATH_TO_AUDIO, '*.wav'))
#print(audio_files)

For testing use a test file randomly extracted from one of the audio files

In [11]:
# set a single pregenerated file for testing
TEST_FILE = ['processed_0428.wav']

## Transcription
Load and instantiate a pre-trained model from NVIDIA. In this case we use CTC decoding as output. 

In [12]:
# load model once
model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_de_quartznet15x5")

[NeMo I 2022-06-09 13:47:38 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.
[NeMo I 2022-06-09 13:47:38 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo
[NeMo I 2022-06-09 13:47:38 common:747] Instantiating model from pre-trained checkpoint


[NeMo W 2022-06-09 13:47:39 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ä
    - ö
    - ü
    - ß
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-06-09 13:47:39 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    V

[NeMo I 2022-06-09 13:47:39 features:259] PADDING: 16
[NeMo I 2022-06-09 13:47:39 features:276] STFT using torch


[NeMo W 2022-06-09 13:47:40 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    


[NeMo I 2022-06-09 13:47:41 save_restore_connector:209] Model EncDecCTCModel was successfully restored from /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.


Retrieve the transcript of the given audio sequence

In [13]:
stt_output = get_stt_transcription(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
print(stt_output)

dauer mühe men ana lorke dar sich der ehnhalt für den burhstulatkort jenitem kulturs und ins sgortom di gonzi praiti und sich aniüber holtort wisedine die löcher und die elekriditemenu alwende bundesrout od otrdeaige eitgenössische partlament sigar wie deine herte fan holo krieger beschlossen het wage demhämme genauner rlichtel löcher und lücken purzfölle sind es ga di umbriffig und der emesaijascabia culumdi lösigen im rahmen für eine quasu mushohent natürlich gisde pflich krantsbereich wommeche amol eini sa gocha zu gunscht aber weiger dem faltiksächslich grundlag grich für wide gehen die substantielliu unterstütz und wer bemeint das gretizkapix für vororder film jetz nimmen die gongsetzer des genes long ja ic haute das der bond behaltert als trennte tatieandere countag ziemit derrititerator setzmache dock abe über ei mer wegcremmnisse berlegt aber bem me nito colcordnytramnatürichdarßas formelder grund vorschiebe unterumgetzs noch docordon idiseme mönknorie analoge und das horhsctul

For comparison we attach the actual true transcript of the given sequence

In [15]:
#true_transcript = "die zur Verfügung stehenden Mittel fair aufgeteilt werden, um sämtliche Unternehmen unterstützen zu können, die jetzt dringend unsere Hilfe brauchen. Zu erwähnen ist auch, dass bei vielen betroffenen Unternehmen die Personalkosten durch die Kurzarbeitsentschädigung oder durch die Corona-Erwerbsersatzentschädigung seit März 2020 gedeckt"

In [16]:
#print(true_transcript)

In [17]:
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable


## TODO
Implement the start and end point detection for each sentence, then pass it along to the sentence aligner

Get the probabilities for each token in the transcript

In [18]:
def get_stt_probs(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    probs = softmax(logits)
    return probs

def get_stt_logits(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    return logits

def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

In [19]:
stt_probs = get_stt_probs(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
stt_logits = get_stt_logits(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
#print(stt_logits)

## Timestep extraction
Define the duration of a timestep 

In [22]:
# 40ms is duration of a timestep at output of the model
time_stride = 0.02

Get the model's alphabet and replace unknown token with 'space'

In [23]:
labels = list(model.cfg.decoder.vocabulary) + ['blank']
labels[0] = 'space'

In [24]:
print(labels)

['space', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'ö', 'ü', 'ß', 'blank']


In [25]:
try:
    from plotly import graph_objects as go
except ModuleNotFoundError:
    !pip install plotly
    from plotly import graph_objects as go

In [26]:
'''
# plot probability distribution over characters for each timestep
fig_probs = go.Figure(
    go.Heatmap(z=stt_probs.transpose(),
               colorscale=[
                   [0, 'rgb(30,62,62)'],
                   [1, 'rgb(30,255,30)'],
               ],
               y=labels,
               dx=time_stride,
               name='Probs',
               hovertemplate='Time: %{x:.2f} s<br>Character: %{y}<br>Probability: %{z:.2f}<extra></extra>'),
    layout={
        'height': 300,
        'xaxis': {'title': 'Time, s'},
        'yaxis': {'title': 'Characters'},
        'title': 'Character Probabilities',
        'margin': dict(l=0, r=0, t=40, b=0, pad=0),
    }
)
fig_probs.show()
'''

"\n# plot probability distribution over characters for each timestep\nfig_probs = go.Figure(\n    go.Heatmap(z=stt_probs.transpose(),\n               colorscale=[\n                   [0, 'rgb(30,62,62)'],\n                   [1, 'rgb(30,255,30)'],\n               ],\n               y=labels,\n               dx=time_stride,\n               name='Probs',\n               hovertemplate='Time: %{x:.2f} s<br>Character: %{y}<br>Probability: %{z:.2f}<extra></extra>'),\n    layout={\n        'height': 300,\n        'xaxis': {'title': 'Time, s'},\n        'yaxis': {'title': 'Characters'},\n        'title': 'Character Probabilities',\n        'margin': dict(l=0, r=0, t=40, b=0, pad=0),\n    }\n)\nfig_probs.show()\n"

Extract timestamps for 'space' tokens

In [27]:
#print(np.argmax(stt_probs[6]))

In [28]:
#print(stt_probs[0])

In [29]:
#for i in range(0, stt_probs.shape[0]):
#    print(np.argmax(stt_probs[i]))

In [30]:
# get timestamps for space symbols
spaces = []

state = ''
idx_state = 0

if np.argmax(stt_probs[0]) == 0:
    state = 'space'

for idx in range(1, stt_probs.shape[0]):
    current_char_idx = np.argmax(stt_probs[idx])
    if state == 'space' and current_char_idx != 0 and current_char_idx != 28:
        spaces.append([idx_state, idx-1])
        state = ''
    if state == '':
        if current_char_idx == 0:
            state = 'space'
            idx_state = idx

if state == 'space':
    spaces.append([idx_state, len(stt_probs)-1])

In [31]:
#print(spaces)

In [32]:
#print(stt_output)

split audio signal into separate words

In [33]:
from IPython.display import Audio, display

In [34]:
#signal, sample_rate = librosa.load(TEST_FILE[0], sr=None)

In [35]:
#print(stt_output)

In [36]:
storage_words = list()
storage_start_time = list()
storage_end_time = list()

In [37]:
# calibration offset for timestamps: 180 ms
offset = -0.18

# split the transcript into words
words = stt_output.split()

# cut words
pos_prev = 0
for j, spot in enumerate(spaces):
    display(words[j])
    pos_end = offset + (spot[0]+spot[1])/2*time_stride
    #display(Audio(signal[int(pos_prev*sample_rate):int(pos_end*sample_rate)],
    #             rate=sample_rate))
    storage_words.append(words[j])
    storage_start_time.append(pos_prev)
    storage_end_time.append(pos_end)
    pos_prev = pos_end

#display(words[j+1])
#display(Audio(signal[int(pos_prev*sample_rate):],
#        rate=sample_rate))


'dauer'

'mühe'

'men'

'ana'

'lorke'

...

'mit'

'siburtaibliebbezigxond'

'dunke'

'fürma'

IndexError: list index out of range

In [38]:
print(len(storage_start_time))

2693


In [39]:
processed_stt_output = pd.DataFrame(
    {'startTime': storage_start_time,
     'endTime': storage_end_time,
     'words': storage_words
    })

In [40]:
print(processed_stt_output)

      startTime  endTime                   words
0          0.00     0.55                   dauer
1          0.55     0.82                    mühe
2          0.82     1.03                     men
3          1.03     1.28                     ana
4          1.28     2.07                   lorke
...         ...      ...                     ...
2688    1660.93  1661.18             chlossstehe
2689    1661.18  1662.15                     mit
2690    1662.15  1662.33  siburtaibliebbezigxond
2691    1662.33  1662.94                   dunke
2692    1662.94  1663.18                   fürma

[2693 rows x 3 columns]


## Language Model
Get the 5-gram german language model from huggingface (https://huggingface.co/aware-ai/wav2vec2-xls-r-1b-5gram-german)

In [41]:
lm_path = "language_model/wav2vec2-xls-r-1b-german-german-5g.binary"

Instantiate beam search with language model

In [42]:
beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
    vocab=list(model.decoder.vocabulary),
    beam_width=16,
    alpha=2, beta=1.5,
    lm_path=lm_path,
    num_cpus=max(os.cpu_count(), 1),
    input_tensor=False)

In [43]:
best_sentence_prediction = beam_search_lm.forward(log_probs = np.expand_dims(stt_probs, axis=0), log_probs_length=None)[0][0][1]

In [44]:
print(best_sentence_prediction)

dauer mühen analoge dersich der inhalt für den buchstaben item kulturund in cartolina zi breit und sich nierozci und die elektrifizierenden bundesrat oddiygina partalen sogar wie eine härtefall kriege geschlossene wadenhoch und lücken purzlen es gadi um briti und der mesicu ti lesigen im rahmen für eine quaschuhe natürlich gesteinstechnische zu wunscht aber weggedampft sächlich grundlag glich für wideeyed die substantielle unterstützden gemeint das gletscherfreie jetzt nimmer die gangster des genes long jachtaanbod behaltet als trennte datierter contestants doable umnitsy aber bemeiern drammatica formeller grund vorschieben untergeordneten reaaaally und das hochschulartspezifisch danke woliger e rotmarderaquarellpinsel da und herragenden knitwear waermekissen erstaunst gewarnt worden sind jahreshälfte es kidner kulturhaus wie so das megalithisches entschädigung zum teilrenovieren betroffen ichwarsnicht besiedelten stadt bibbert tausende frankonia und die gigarec sind fencing mationspro

## Alignment Procedure

In [45]:
from aligning.sentence_aligner import SentenceAligner
from aligning.stt_reader import NeMoSttOutputReader
from aligning.utils import Word

In [46]:
final_stt_output = []
for i in range(0, len(processed_stt_output)):
    final_stt_output.append(Word(storage_words[i], storage_start_time[i], storage_end_time[i]))

#print(final_stt_output)

In [47]:
#print(truth_sentences)

In [48]:
sentence_aligner = SentenceAligner()

In [49]:
sentence_alignment, _ = sentence_aligner.predict_one(
    truth_sentences, final_stt_output,
    do_length_ratio_full_transcript_filtering=False,
    do_time_correction=False,
)

In [50]:
print(sentence_alignment[0])

In [51]:
NONE_TIME_STRING_REPRESENTATION = '-1.0'
def transform_start_time(time):
    if time is None:
        return NONE_TIME_STRING_REPRESENTATION
    else:
        return time

def transform_end_time(time):
    if time is None:
        return NONE_TIME_STRING_REPRESENTATION
    else:
        return time

In [53]:
with open("test.txt", 'w', encoding='utf-8') as f:
        for aligned_sentence in sentence_alignment:
            f.write(f'{transform_start_time(aligned_sentence.start_time)}\t{transform_end_time(aligned_sentence.end_time)}\t{aligned_sentence.sentence}\n')

In [52]:
print(sentence_alignment[0].end_time)

0.55
